In [1]:
import pandas as pd
import numpy as np
from faker import Faker
from collections import defaultdict
from sqlalchemy import create_engine
import datetime
import random
import re


# Fake data creation

In [2]:
from numpy import integer

Faker.seed(50)
fake=Faker()

money_transaction = defaultdict(list)

row_number=50



In [7]:
# Creating initiated_account_id as much as the given input
bank_info={"BankAnk":"100","BankIST":"200","BankBUR":"300"}

for i in range(row_number):
    money_transaction['initiated_account_id'].append(random.choice(list(bank_info.values()))+fake.bothify(text='####'))

initiated_account_id=money_transaction["initiated_account_id"]

#Creating other futures
initiated_account_balance_old=[]
amount=[]
initiated_account_balance_new=[]
account_id=[]
account_balance=[]
timestamp=[]

In [8]:
for idx in range(len(money_transaction['initiated_account_id'])): #For loop for creating random values in features

    random_range=[random.randint(10,2000),random.randint(2000,4000),random.randint(4000,6000)]
    temp_initiated_account_balance=[random_range]
    temp_initiated_account_balance_new=[]
    temp_account_id_list=[]
    temp_amount_list=[random.randrange(10,temp_initiated_account_balance),]
    temp_timestamp_list=[]
    

    #Creating random relationship on two nodes
    for j in range(random.randint(1,15)): 
        
        #Choosing random initiated_account_id and adding to reciever_id
        rnd_account_id=''.join(random.choices(initiated_account_id)) 
        
        #If sender and receiver feature value same break from the loop
        if (initiated_account_id[idx]== rnd_account_id):
            continue

        if len(temp_initiated_account_balance)>= j:
                break
#---------------------------------------------------------------------
        temp_amount_list.append(random.randrange(10,temp_initiated_account_balance[j]))
        if temp_initiated_account_balance[j] - temp_amount_list[j] >= 0 :
                temp_initiated_account_balance.append(temp_initiated_account_balance[j] - temp_amount_list[j])

        #Creating the new balance after the transaction
        #temp_initiated_account_balance_new.append(temp_initiated_account_balance[j]-temp_amount_list[j])
#-------------------------------------------------------------------------
        temp_account_id_list.append(rnd_account_id)
        temp_timestamp_list.append(fake.date_time_ad(start_datetime=datetime.datetime(2022,9,14)))    
    #Check the size of the receiver_id compare it with initiated_account_id
    if len(account_id)==len(money_transaction['initiated_account_id']):
            break
    print(temp_amount_list)


    #Appending every feature value to list
    account_id.append(temp_account_id_list)
    amount.append(temp_amount_list)
    initiated_account_balance_old.append(temp_initiated_account_balance)
    initiated_account_balance_new.append(temp_initiated_account_balance_new)
    timestamp.append(temp_timestamp_list)
        

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'

In [ ]:
#Ejecting fake/manipulated data do Dataframe
money_transaction['initiated_account_balance(old)']=initiated_account_balance_old
money_transaction['amount']=amount
money_transaction['initiated_account_balance(new)']=initiated_account_balance_new
money_transaction['account_id']=account_id
money_transaction['timestamp']=timestamp
print(len(money_transaction['amount']))
df=pd.DataFrame(money_transaction)
df

# Neo4j

In [ ]:
from neo4j import GraphDatabase

In [ ]:
transaction_info=df.values.tolist()

In [ ]:
uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "test"))

#Deleting existing values and relationships
def clear_db(tx):
    tx.run("MATCH(n) Detach Delete n")
#Creating customer node
def create_customer(tx, customer_id):
    tx.run("CREATE (c1:customer {name: $name, customer_id: $id})",name='customer', id=customer_id)

#Creating relationship between customer nodes
def create_transaction(tx,customer_id,other_customer_id,amount,timestamp):
    tx.run("MATCH (c1:customer),(c2:customer) "
    "Where c1.customer_id= $id and c2.customer_id= $otherid "
    "Create (c1)-[t:Transaction{amount: $amount, time: $timestamp}]->(c2)",
    id=customer_id,otherid=other_customer_id,amount=amount,timestamp=timestamp)


with driver.session() as session:

    session.execute_write(clear_db)

    for i in range(len(transaction_info)):
        session.write_transaction(create_customer, str(transaction_info[i][0]))

    deneme=[]    
    for element in transaction_info:
        for block in range(1,len(element)):
            for j in range(len(element[block])):
                session.write_transaction(create_transaction,element[0],element[1][j],element[2][j],element[3][j])
            if element[1][j]==len(element[block]):
                    break
        print(j)


driver.close()